In [0]:
import os
import dlib
import numpy
from skimage.io import imread,imsave
import cv2
import numpy as np
from matplotlib import pyplot as plt

**Mounting the google drive to hosted runtime.**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
t=!ls "/content/gdrive/My Drive"

**Unzipping the contents of the IMFDB Face Recognition Data from the hosted runtime.**

In [0]:
!unzip -q "/content/gdrive/My Drive/IMFDB_final.zip"

**Getting the Eyes, Ears and Nose x-y coordinates as a template for face detection and using min-max scaling on the same so that the coordinates are uniformly distributed in the range (0,1).**

In [0]:
OUTER_EYES_AND_NOSE = [36, 45, 33]
TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)

**Face detection on IMFDB Face Dataset.**  



*   Walking through the training images for pre-processing.
*   Resizing the image to 160 x 160.
*   Using the dlib frontal detector with 68 point facial landmarks.
*   Using affine transformation of cv2 library for aligning the image and resizing it to 96 x 96.
*   The above pre-processed images are saved to the output directory with the same tree structure as that of the original dataset.



In [0]:
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import skimage.transform
input_dir="/content/IMFDB_final"
output_dir="Train/"
output_image="Images/"
image_label=list()
count=0
thumb=-1
for root, dirs, files in os.walk(input_dir):
	  for file in files:
		    if file.endswith(".jpg"):
			      print(file)
			      img_path=os.path.join(root, file)
			      img_folder=img_path.split('/')[3]
			      img=cv2.imread(img_path)
		        #img_gray=rgb2gray(img)
			      img_resized = cv2.resize(img, (160,160), interpolation = cv2.INTER_AREA)
		        #plt.imshow(x)
			      detector=dlib.get_frontal_face_detector()
			      dets=detector(img_resized)
			      if len(dets)>1 or len(dets)==0:
				      bound_box=-1
			      else:
				      bound_box=dets[0]
			      if numpy.sum(bound_box)==-1:
				      thumb=-1
			      else:
			        model_path="/content/gdrive/My Drive/face_landmarks.dat"
			        predictor=dlib.shape_predictor(model_path)
			        pnts=predictor(img_resized,bound_box)
			        landmarks=np.float32(list(map(lambda p:(p.x,p.y),pnts.parts())))
			        landmarksindices=np.array(OUTER_EYES_AND_NOSE)
			        affine_transform=cv2.getAffineTransform(landmarks[landmarksindices],\
														96*MINMAX_TEMPLATE[landmarksindices])
			        thumb=cv2.warpAffine(img_resized,affine_transform,(96,96))
			        if(len(image_label)==0):
			          image_label.append(img_folder)
			          name=file.rsplit('_', 1)[0]+"_"+str(count)+'.jpg'
			          count=count+1
			        else:
				        if(file.rsplit('_', 1)[0]==image_label[-1]):
				          name=file.rsplit('_', 1)[0]+"_"+str(count)+'.jpg'
				          count=count+1
				        else:
				          count=0
				          image_label.append(img_folder)
				          name=file.rsplit('_', 1)[0]+"_"+str(count)+'.jpg'
				          count=count+1
		    if(numpy.sum(thumb)==-1):
			    continue
		    
		    out_dir1=os.path.join(output_image)			
		    out_dir=os.path.join(output_dir,img_folder)
		    if not os.path.exists(out_dir1):
			    os.makedirs(out_dir1)
		    if not os.path.exists(out_dir):
			    os.makedirs(out_dir)
		    out_path1=os.path.join(output_image,name)
		    out_path=os.path.join(output_dir,file.rsplit('_', 1)[0]+"/",name)
		    cv2.imwrite(out_path1,thumb)
		    cv2.imwrite(out_path,thumb)

Srihari_20.jpg
Srihari_150.jpg
Srihari_32.jpg
Srihari_229.jpg
Srihari_45.jpg
Srihari_103.jpg
Srihari_218.jpg
Srihari_79.jpg
Srihari_17.jpg
Srihari_44.jpg
Srihari_23.jpg
Srihari_231.jpg
Srihari_90.jpg
Srihari_18.jpg
Srihari_201.jpg
Srihari_3.jpg
Srihari_158.jpg
Srihari_5.jpg
Srihari_139.jpg
Srihari_146.jpg
Srihari_52.jpg
Srihari_164.jpg
Srihari_185.jpg
Srihari_97.jpg
Srihari_94.jpg
Srihari_219.jpg
Srihari_110.jpg
Srihari_130.jpg
Srihari_61.jpg
Srihari_29.jpg
Srihari_195.jpg
Srihari_217.jpg
Srihari_64.jpg
Srihari_125.jpg
Srihari_65.jpg
Srihari_38.jpg
Srihari_163.jpg
Srihari_33.jpg
Srihari_239.jpg
Srihari_123.jpg
Srihari_98.jpg
Srihari_48.jpg
Srihari_54.jpg
Srihari_225.jpg
Srihari_161.jpg
Srihari_31.jpg
Srihari_12.jpg
Srihari_166.jpg
Srihari_214.jpg
Srihari_145.jpg
Srihari_155.jpg
Srihari_4.jpg
Srihari_207.jpg
Srihari_106.jpg
Srihari_59.jpg
Srihari_234.jpg
Srihari_86.jpg
Srihari_223.jpg
Srihari_156.jpg
Srihari_1.jpg
Srihari_43.jpg
Srihari_224.jpg
Srihari_28.jpg
Srihari_66.jpg
Srihari_242.

In [0]:
!zip -r /content/Images.zip /content/Images

updating: content/Images/ (stored 0%)
  adding: content/Images/AkshayKumar_2.jpg (deflated 4%)
  adding: content/Images/SureshGopi_0.jpg (deflated 5%)
  adding: content/Images/PareshRaval_10.jpg (deflated 4%)
  adding: content/Images/PremaNazir_6.jpg (deflated 5%)
  adding: content/Images/Thilakan_35.jpg (deflated 5%)
  adding: content/Images/AkshayKumar_4.jpg (deflated 5%)
  adding: content/Images/VinodKhanna_67.jpg (deflated 5%)
  adding: content/Images/Mammukoya_1.jpg (deflated 6%)
  adding: content/Images/KarunaBenerjee_77.jpg (deflated 5%)
  adding: content/Images/Ranimukherji_123.jpg (deflated 5%)
  adding: content/Images/PareshRaval_31.jpg (deflated 4%)
  adding: content/Images/Simran_253.jpg (deflated 4%)
  adding: content/Images/Srihari_20.jpg (deflated 4%)
  adding: content/Images/Dileep_47.jpg (deflated 4%)
  adding: content/Images/AkshayKumar_76.jpg (deflated 5%)
  adding: content/Images/Nagarjuna_68.jpg (deflated 5%)
  adding: content/Images/JagapathiBabu_42.jpg (deflated 

**Archiving the contents of the output directory for model training and embedding computation.Link is mentioned inline.**  
[FaceRecognition_Architecture_ProposedTripletLoss](https://drive.google.com/open?id=1rll6clYGBVmC5ZBvFuGTDduJXWQotgvl)



In [0]:
!zip -r /content/Train.zip /content/Train/


  adding: content/Train/ (stored 0%)
  adding: content/Train/Srihari/ (stored 0%)
  adding: content/Train/Srihari/Srihari_20.jpg (deflated 4%)
  adding: content/Train/Srihari/Srihari_32.jpg (deflated 4%)
  adding: content/Train/Srihari/Srihari_45.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_79.jpg (deflated 4%)
  adding: content/Train/Srihari/Srihari_17.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_44.jpg (deflated 4%)
  adding: content/Train/Srihari/Srihari_23.jpg (deflated 4%)
  adding: content/Train/Srihari/Srihari_36.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_18.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_3.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_5.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_52.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_61.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_29.jpg (deflated 5%)
  adding: content/Train/Srihari/Srihari_64.jpg (deflated 4%)
  add